In [1]:
import numpy as np 
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from numba import jit
import json

In [2]:
df = pd.read_csv('../datelets/datalet2.csv',index_col=False)

In [3]:
headers = []

for i in range(df.shape[1]):
    headers.append('period{}'.format(i))
 

df.columns = headers    

display(df)

,period0,period1
0,2.0,3.0
1,3.0,2.0
2,2.0,1.0
3,1.0,3.0
4,3.0,2.0
...,...,...
11999993,2.0,2.0
11999994,2.0,1.0
11999995,1.0,2.0
11999996,2.0,2.0


In [4]:
counts = {}
for x in range(len(df['period0'].value_counts())):
    counts['{}.0'.format(x)]= df['period0'].value_counts()[x]
    
print(counts)


{'0.0': 3000673, '1.0': 2999992, '2.0': 2999529, '3.0': 2999804}


In [5]:
df['ydata'] = df['period1']
output = pd.pivot_table(data=df, index=['period0'], columns=['period1'], values='ydata', aggfunc='count')
display(output)



period1,0.0,1.0,2.0,3.0
period0,,,,
0.0,751350,750188,749427,749708
1.0,747648,750634,750553,751157
2.0,751349,749149,749909,749122
3.0,750326,750022,749639,749817


In [6]:
#This is a the stochastic markovchain that tells you, if you start at a perticular MEV state, where might you end up. the matrix is transposed to be listed in the traditional shape
#of a markov chain transition matrix

tmatrix = output.transpose(copy=False)

print(counts['1.0'])

for x in range(len(counts)):
    tmatrix[x] = (tmatrix[x] / counts['{}.0'.format(x)])

display(tmatrix)



2999992


period0,0.0,1.0,2.0,3.0
period1,,,,
0.0,0.250394,0.249217,0.250489,0.250125
1.0,0.250007,0.250212,0.249756,0.250024
2.0,0.249753,0.250185,0.250009,0.249896
3.0,0.249847,0.250386,0.249747,0.249955
